In [22]:
import pandas as pd

In [23]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [24]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [25]:
len(data)

174643

In [26]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [27]:
# hard-coded awards
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [28]:
import re

In [130]:
def reduce(line):
    # remove words "best", "performance", "motion", "picture", "limited", "series", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bmotion\b|\bpicture\b|\blimited\b|\bseries\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
for award in awards:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['drama']
['musical', 'or', 'comedy']
['actress', 'drama']
['actor', 'drama']
['actress', 'musical', 'or', 'comedy']
['actor', 'musical', 'or', 'comedy']
['actress', 'supporting']
['actor', 'supporting']
['director']
['screenplay']
['animated']
['foreign']
['original', 'score']
['original', 'song']
['television', 'drama']
['television', 'musical', 'or', 'comedy']
['television', 'or']
['actress', 'or', 'television']
['actor', 'or', 'television']
['actress', 'television', 'drama']
['actor', 'television', 'drama']
['actress', 'television', 'musical', 'or', 'comedy']
['actor', 'television', 'musical', 'or', 'comedy']
['actress', 'supporting', 'or', 'television']
['actor', 'supporting', 'or', 'television']
['cecil', 'demille']


In [131]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words 
        else:
            award_num_keywords_map[i] = len(award)
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

In [132]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

[(0, 1),
 (1, 1),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 2),
 (14, 2),
 (15, 2),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 3),
 (20, 3),
 (21, 3),
 (22, 3),
 (23, 2),
 (24, 2),
 (25, 2)]

In [133]:
awards_reduced

[['drama'],
 ['musical', 'comedy'],
 ['actress', 'drama'],
 ['actor', 'drama'],
 ['actress', 'musical', 'comedy'],
 ['actor', 'musical', 'comedy'],
 ['actress', 'supporting'],
 ['actor', 'supporting'],
 ['director'],
 ['screenplay'],
 ['animated'],
 ['foreign'],
 ['original', 'score'],
 ['original', 'song'],
 ['television', 'drama'],
 ['television', 'musical', 'comedy'],
 ['television'],
 ['actress', 'television'],
 ['actor', 'television'],
 ['actress', 'television', 'drama'],
 ['actor', 'television', 'drama'],
 ['actress', 'television', 'musical', 'comedy'],
 ['actor', 'television', 'musical', 'comedy'],
 ['actress', 'supporting', 'television'],
 ['actor', 'supporting', 'television'],
 ['cecil', 'demille']]

In [144]:
'actress' not in awards_reduced[2]

False

In [135]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [136]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [137]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [138]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [139]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [140]:
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line)
    line = remove_at(line)
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

In [141]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [142]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[entity.text]=[entity.label_]
    return tags

In [123]:
# some ideas to improve spaCy's Name Entity Recognizer
# https://github.com/explosion/spaCy/issues/1655
# https://github.com/explosion/spacy/blob/master/examples/training/train_new_entity_type.py
# https://spacy.io/usage/training#section-ner

In [93]:
%%time
# test
num = 0
count = 0
pattern = re.compile(r'amy adams')
for line in cleansed_data:    
    match = re.findall(pattern, line.lower())                  
    if match:
        tags = identify_entities(line)
        count += 1
        # check the first 10 cases
        if count < 10:
            print(num)
            print(match)
            print(tweet)
            print(tags.keys())
            print()
    num += 1
    
print(count)

295
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['Amy Adams   ', '3'])

1240
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['Amy Adams', '  '])

1619
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['  Amy Adams', 'Marchesa'])

1947
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['   Amy Adams', 'Marchesa', 'Xox'])

1957
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['Da Amy Adams', '      '])

2298
['amy adams']
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's sayi

In [153]:
def find_award_winner(awards_reduced, award_index, verbose=False):
    print('Predicting for:', awards[award_index])    
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match other than 'best':", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"
    pattern = re.compile(r'(\bwin)|(\bwon\b)|(\bbest\b)', re.IGNORECASE)
#     pattern = re.compile(r'(\bnominate)|(\bnominee\b)|(\bwin)|(\bwon\b)|(\bbest\b)|(\b{0}\b)'.format(award[1]), re.IGNORECASE)
    entity_dict = {}
    
    num = 0
    for line in cleansed_data:
        match = re.findall(pattern, line.lower())
        # if line contains at least number keywords to match and pattern to be matched is found
        if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
            tags = identify_entities(line)

            if verbose:
                # print the first 5 occurrences
                if num < 5:
                    print(match)
                    print(tweet)
                    print(tags.keys())
                    print()
            
            # reduce entities in tags
#             entities = []
#             if 'actor' in award or 'actress' in award or 'director' in award:
#                 for entity in tags.keys():
#                     if tags[entity] == 'PERSON':
#                         entities.append(entity)
#             elif 'cecil' not in award:
#                 for entity in tags.keys():
#                     if tags[entity] != 'PERSON':
#                         entities.append(entity)
                        
            for entity in tags.keys():
                entity = entity.strip() 
                if len(entity) > 1:    
                    if entity not in entity_dict:
                        entity_dict[entity] = 1
                    else:
                        entity_dict[entity] += 1
            num += 1
            
    print('num of matches:', num)
    return entity_dict

In [154]:
%%time
# test
entity_dict = find_award_winner(awards_reduced, 0, verbose=True)

Predicting for: Best Motion Picture - Drama
award name reduced: ['drama']
num keywords to match other than 'best': 1
[('', '', 'best')]
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['Lincoln for Best Movie Drama  Silver Linings Playbooks for Comedy Musical  Homeland for Tv'])

[('', '', 'best')]
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['  ', ' Christof Waltz'])

[('win', '', ''), ('', '', 'best')]
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys(['Maggie Smith', ' Go', 'McGonnagall'])

[('', '', 'best')]
Says @BenAffleck: "I also didn't get the Acting Nomination...no one's saying I got snubbed there!" #pressroom #GoldenGlobes
dict_keys([])

[('win', '', ''), ('', 'won', ''), ('', '', 'best')]
Says @BenAffleck: "I also didn

In [158]:
from fuzzywuzzy import fuzz

In [156]:
def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

# Predictions for Award Winner

In [165]:
%%time
for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_winner(awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]
    # filter for names if necessary
    if 'actor' in award or 'actress' in award or 'director' in awards_reduced[key]:
        top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    print()

Predicting for: Best Motion Picture - Drama
award name reduced: ['drama']
num keywords to match other than 'best': 1
num of matches: 810
top results:
[('Homeland', 164), ('Jessica Chastain', 79), ('Daniel Day Lewis', 62), ('Lincoln', 61), ('Claire Danes', 59), ('Zero', 44), ('Dark Thirty', 39), ('Argo', 39), ('first', 34), ('Damien Lewis', 30), ("Les Miz'", 29), ('Lewis  Chastain', 29), ('Damian Lewis', 25), ('night', 24), ('15', 22), ('Ben Affleck', 20), ('for', 13), ('Zero Dark Thirty', 12), ('Kathryn Bigelow', 10), ('Latest', 10), ('Breaking', 9), ('12 year old', 9), ('Best', 8), ('Damian', 8), ('A lanky', 8), ('Lewis', 7), ('Breaking Bad', 7), ('Best TV', 6), ('WINNER', 6), ('RT', 6), ('Anyone', 6), ('Julia Roberts', 6), ('Les Miserables', 6), ('CNN', 5), ('Homeland Best', 4), ('First', 4), ('Congrats', 4), ('3rd', 3), ('Best Actor', 3), ('Showtime', 3), ('Oscar', 3), ('Motion Picture', 3), ('Chastain', 3), ('Ben', 3), ('Congrats to', 3), ('this year', 2), ('Series', 2), ('TV', 2),

num of matches: 219
top results:
[('Christoph Waltz', 153), ('Django', 28), ('Ed Harris', 27), ('Congratulations', 11), ('RJJ', 11), ('Unchained', 10), ('Game Change', 8), ('Django Unchained', 6), ('Christopher Waltz', 3), ('Great', 3), ('Alan Arkin', 2), ('Waltz', 2), ('Leo', 2), ('Awards', 2), ('Tommy Lee Jones', 2), ('Piper Laurie', 2), ('Twin Peaks', 2), ('1991', 2), ('Ed Harris  Game Change', 2), ('WINNER', 2), ('Let Schmidt', 2), ('Les Miserable', 2), ("American Splendor'", 2), ('Modern Family', 1), ('Eric Stonestreet', 1), ('Cam', 1), ('Go', 1), ('ARGO', 1), ('Christof Waltz', 1), ('Christopher Waltz Wins', 1), ('Leonardo Di Caprio', 1), ('Christopher Walz', 1), ('El', 1), ('Leo DiCaprio', 1), ('Cristoph Waltz', 1), ('Really', 1), ('Niceeee', 1), ('DiCaprio', 1), ('the 1st award', 1), ('tonight', 1), ('Maggie Smith', 1), ('Downton Abbey', 1), ('Inglorious Basterds', 1), ('Bueh', 1), ('Harrison Ford', 1), ('Lol', 1), ('Marin', 1), ('Christoph', 1), ('RT', 1), ('Nachos', 1)]
top r

num of matches: 4
top results:
[('Homeland', 5), ('Showtime', 2), ('Claire Danes', 1)]
top results after merging:
[('Homeland', 5), ('Showtime', 2), ('Claire Danes', 1)]

Predicting for: Best Television Series - Musical or Comedy
award name reduced: ['television', 'musical', 'comedy']
num keywords to match other than 'best': 2
num of matches: 557
top results:
[('Les Miserables', 133), ('Hugh Jackman', 131), ('Lena Dunham', 54), ('Jennifer Lawrence', 53), ('Girls', 46), ('Don Cheadle', 36), ("Les Miserables'", 24), ('Eagles', 18), ('Bradley Cooper', 18), ('VW', 18), ('House of Lies', 14), ('Congrats', 12), ('Best Actor', 11), ('Les Mis', 10), ('WINNER', 10), ('Les', 10), ('Best Motion Picture', 8), ('Lawrence', 7), ('Best Actress', 7), ('night', 6), ('Bill Murray', 6), ('Miserables', 6), ('CNN', 5), ('Meryl', 4), ('HBO', 4), ("'Girls'", 4), ('Argo', 4), ('Meryl Streep', 3), ('RT', 3), ('Silver Linings Playbook', 3), ('House', 3), ('Best', 3), ('Silver Linin', 3), ('JLAW', 2), ('tonight'

num of matches: 219
top results:
[('Christoph Waltz', 153), ('Django', 28), ('Ed Harris', 27), ('Congratulations', 11), ('RJJ', 11), ('Unchained', 10), ('Game Change', 8), ('Django Unchained', 6), ('Christopher Waltz', 3), ('Great', 3), ('Alan Arkin', 2), ('Waltz', 2), ('Leo', 2), ('Awards', 2), ('Tommy Lee Jones', 2), ('Piper Laurie', 2), ('Twin Peaks', 2), ('1991', 2), ('Ed Harris  Game Change', 2), ('WINNER', 2), ('Let Schmidt', 2), ('Les Miserable', 2), ("American Splendor'", 2), ('Modern Family', 1), ('Eric Stonestreet', 1), ('Cam', 1), ('Go', 1), ('ARGO', 1), ('Christof Waltz', 1), ('Christopher Waltz Wins', 1), ('Leonardo Di Caprio', 1), ('Christopher Walz', 1), ('El', 1), ('Leo DiCaprio', 1), ('Cristoph Waltz', 1), ('Really', 1), ('Niceeee', 1), ('DiCaprio', 1), ('the 1st award', 1), ('tonight', 1), ('Maggie Smith', 1), ('Downton Abbey', 1), ('Inglorious Basterds', 1), ('Bueh', 1), ('Harrison Ford', 1), ('Lol', 1), ('Marin', 1), ('Christoph', 1), ('RT', 1), ('Nachos', 1)]
top r